In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import random
import pickle
from sklearn.svm import SVC

print("Libraries imported successfully.")

# Section 2: Preprocessing and Graph Construction

# File paths
original_features_path = "../foursquare_data/features.json"
original_edges_path = "../foursquare_data/edges.csv"

# Load data
def load_data():
    print("Loading original features and edges...")
    with open(original_features_path, "r") as f:
        features = json.load(f)
    print(f"Loaded {len(features)} nodes with features.")
    
    edges = pd.read_csv(
        original_edges_path, names=["source", "target"], skiprows=1
    )  # Skip header row
    print(f"Loaded {len(edges)} edges.")
    return features, edges

# Create graph and extract largest connected component
def create_graph(features, edges):
    print("Creating graph from edges...")
    edges["source"] = edges["source"].astype(str)
    edges["target"] = edges["target"].astype(str)
    G = nx.from_pandas_edgelist(edges, source="source", target="target")
    print(f"Graph created with {len(G.nodes)} nodes and {len(G.edges)} edges.")

    print("Identifying the largest connected component...")
    largest_cc = max(nx.connected_components(G), key=len)
    G_lcc = G.subgraph(largest_cc).copy()
    print(f"Largest connected component has {len(G_lcc.nodes)} nodes and {len(G_lcc.edges)} edges.")

    print("Assigning features to nodes...")
    for node, feats in features.items():
        if node in G_lcc.nodes:
            G_lcc.nodes[node]["features"] = feats
    print("Node features assigned.")
    
    return G_lcc

def standardize_features(G, output_dim=128):
    print("Standardizing features using binarization and PCA...")

    # Extract nodes with features
    nodes_with_features = [
        node for node, feats in nx.get_node_attributes(G, "features").items() if feats
    ]
    print(f"Number of nodes with features: {len(nodes_with_features)}")

    # Extract feature sets
    feature_list = [
        set(G.nodes[node]["features"]) for node in nodes_with_features
    ]
    
    mlb = MultiLabelBinarizer()
    binary_features = mlb.fit_transform(feature_list)
    print(f"Initial feature matrix shape: {binary_features.shape}")

    # Apply PCA if needed
    if binary_features.shape[1] > output_dim:
        print(f"Reducing dimensions to {output_dim} using PCA...")
        pca = PCA(n_components=output_dim)
        reduced_features = pca.fit_transform(binary_features)
        print(f"Feature matrix shape after PCA: {reduced_features.shape}")
    else:
        print(f"No dimensionality reduction needed. Retaining shape {binary_features.shape}")
        reduced_features = binary_features

    # Assign standardized features back to the corresponding nodes
    print("Assigning standardized features back to nodes...")
    for idx, node in enumerate(nodes_with_features):
        G.nodes[node]["features"] = reduced_features[idx]
    
    print("Feature standardization complete.")
    return mlb, pca if binary_features.shape[1] > output_dim else None

# Load, process, and standardize graph
print("Starting graph preprocessing...")
features, edges = load_data()
G = create_graph(features, edges)
mlb, pca = standardize_features(G, output_dim=128)
print(f"Graph preprocessing complete. Final graph has {len(G.nodes)} nodes and {len(G.edges)} edges.")

def check_connectivity_bfs(G):
    print("Performing BFS to ensure all nodes are connected...")
    start_node = next(iter(G.nodes))  # Get an arbitrary starting node
    visited = set()
    queue = [start_node]

    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.add(node)
            queue.extend(neighbor for neighbor in G.neighbors(node) if neighbor not in visited)

    if len(visited) == len(G.nodes):
        print("All nodes are connected. The graph is a single connected component.")
    else:
        print(f"Graph is not fully connected. Only {len(visited)} out of {len(G.nodes)} nodes are reachable.")

check_connectivity_bfs(G)  # Ensure the graph is a single connected component

# Section 3A: Create Feature Vectors

def create_feature_vectors(G, edges):
    print("Creating feature vectors for ML tasks...")
    X, y = [], []

    print("Processing positive samples (existing edges)...")
    for i, (_, row) in enumerate(edges.iterrows()):
        node1, node2 = str(row["source"]), str(row["target"])
        if (
            node1 in G.nodes 
            and node2 in G.nodes 
            and "features" in G.nodes[node1] 
            and "features" in G.nodes[node2]
        ):
            feature_vector = np.array(G.nodes[node1]["features"]) - np.array(G.nodes[node2]["features"])
            X.append(feature_vector)
            y.append(1)
        if i % 50000 == 0:
            print(f"Processed {i} positive samples.")

    print("Generating negative samples (random non-existing edges)...")
    all_nodes = [node for node in G.nodes if "features" in G.nodes[node]]  # Nodes with features
    for i in range(len(edges)):
        node1, node2 = np.random.choice(all_nodes, 2, replace=False)
        if not G.has_edge(node1, node2):
            feature_vector = np.array(G.nodes[node1]["features"]) - np.array(G.nodes[node2]["features"])
            X.append(feature_vector)
            y.append(0)
        if i % 50000 == 0:
            print(f"Generated {i} negative samples.")

    print(f"Feature vectors created. Total samples: {len(X)}")
    return np.array(X), np.array(y)


# Create and split feature vectors
print("Creating and splitting feature vectors...")
X, y = create_feature_vectors(G, edges)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")



/Users/johnkowalczyk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Libraries imported successfully.
Starting graph preprocessing...
Loading original features and edges...
Loaded 30075 nodes with features.
Loaded 701311 edges.
Creating graph from edges...
Graph created with 114324 nodes and 701311 edges.
Identifying the largest connected component...
Largest connected component has 111251 nodes and 699461 edges.
Assigning features to nodes...
Node features assigned.
Standardizing features using binarization and PCA...
Number of nodes with features: 2847
Initial feature matrix shape: (2847, 10141)
Reducing dimensions to 128 using PCA...
Feature matrix shape after PCA: (2847, 128)
Assigning standardized features back to nodes...
Feature standardization complete.
Graph preprocessing complete. Final graph has 111251 nodes and 699461 edges.
Performing BFS to ensure all nodes are connected...
All nodes are connected. The graph is a single connected component.
Creating and splitting feature vectors...
Creating feature vectors for ML tasks...
Processing positi

In [2]:
with open("feature_vectors_svm.pkl", "wb") as f:
    pickle.dump((X, y, X_train, X_test, y_train, y_test), f)

print("Feature vectors and splits saved successfully.")

with open("feature_vectors_svm.pkl", "rb") as f:
    X, y, X_train, X_test, y_train, y_test = pickle.load(f)

print("Feature vectors and splits loaded successfully.")
print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")

# Section 3B: Train the SVM Model
print("Training the SVM model with a linear kernel for interpretability...")
model = SVC(kernel='linear', probability=True, random_state=42)
model.fit(X_train, y_train)

print("Model training complete.")

# Save the trained model
with open("trained_model_svm.pkl", "wb") as f:
    pickle.dump(model, f)
print("SVM model saved to 'trained_model_svm.pkl'.")

# Evaluate the model
print("Evaluating the model on the test set...")
test_accuracy = model.score(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

y_pred = model.predict(X_test)
test_precision = precision_score(y_test, y_pred, zero_division=0)
test_recall = recall_score(y_test, y_pred, zero_division=0)
test_f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1: {test_f1:.4f}")

with open("trained_model_svm.pkl", "rb") as f:
    model = pickle.load(f)
print("SVM model loaded successfully from 'trained_model_svm.pkl'.")

def find_best_nodes(G, neighbors, target_node):
    """
    Find the best nodes among neighbors based on shortest path distance to the target.
    Returns the list of tied 'best' nodes with the minimum distance.
    """
    distances = {
        neighbor: nx.shortest_path_length(G, source=neighbor, target=target_node)
        for neighbor in neighbors
    }
    min_distance = min(distances.values())
    best_nodes = [node for node, dist in distances.items() if dist == min_distance]
    return best_nodes

def predict_next_node(model, G, current_node, target_node, visited, prediction_cache, debug=False):
    """
    Predict the next node from the current node and return step-by-step accuracy.
    Using SVM model now with probability estimates.
    """
    if debug:
        print(f"Predicting next node from current node: {current_node}")

    neighbors = [
        neighbor for neighbor in G.neighbors(current_node)
        if "features" in G.nodes[neighbor] and neighbor not in visited
    ]

    if not neighbors:
        random_choice = random.choice(list(G.neighbors(current_node))) if len(list(G.neighbors(current_node))) > 0 else None
        if debug:
            print("Opting For Random Choice")
        return random_choice, False  # No "best choice" available, random node selected
    
    if target_node in neighbors:
        if debug:
            print(f"Target node {target_node} is a direct neighbor. Automatically selecting it.")
        return target_node, True  # Automatically move to the destination and mark as accurate

    best_nodes = find_best_nodes(G, neighbors, target_node)
    
    # Predict probabilities for neighbors
    predictions = []
    for neighbor in neighbors:
        feature_vector = np.array(G.nodes[neighbor]["features"]) - np.array(G.nodes[target_node]["features"])
        prob = model.predict_proba(feature_vector.reshape(1, -1))[0][1]  # Probability of class '1'
        predictions.append((neighbor, prob))

    # Sort by probability (descending)
    predictions.sort(key=lambda x: x[1], reverse=True)

    if debug:
        print(f"Neighbors were available. Highest predicted probability: {predictions[0][1]:.4f}, Selected Node: {predictions[0][0]}")
    chosen_node = predictions[0][0]  # Node with the highest probability

    is_accurate = chosen_node in best_nodes
    return chosen_node, is_accurate

def find_path(model, G, source, target, max_hops=40, debug=False):
    """
    Find the path while tracking accuracy at each step.
    Outputs path and per-move accuracy data.
    """
    if debug:
        print(f"Starting pathfinding from source: {source} to target: {target}, with max hops: {max_hops}")

    current_node = source
    visited = set()
    prediction_cache = {}
    path = [source]  # Track path regardless of success
    hops = 0
    correct_choices = 0
    step_actuals = []  
    step_predictions = []  

    while hops < max_hops:
        if current_node == target:
            if debug:
                print(f"Pathfinding succeeded: target {target} reached.")
            return path, correct_choices, hops, step_actuals, step_predictions

        visited.add(current_node)
        next_node, is_accurate = predict_next_node(model, G, current_node, target, visited, prediction_cache, debug=debug)

        if not next_node:  # No valid moves
            if debug:
                print(f"Pathfinding failed: no valid neighbors from {current_node}.")
            return path, correct_choices, hops, step_actuals, step_predictions

        # A "best node" always exists
        step_actuals.append(1)  
        step_predictions.append(1 if is_accurate else 0)

        if is_accurate:
            correct_choices += 1

        path.append(next_node)
        current_node = next_node
        hops += 1

    # Failed due to max hops
    if debug:
        print(f"Pathfinding failed: exceeded max hops ({max_hops}).")
    return path, correct_choices, hops, step_actuals, step_predictions

def evaluate_pathfinding(model, G, max_hops=20, num_runs=20):
    """
    Evaluate the pathfinding algorithm, calculate metrics, and print paths.
    """
    total_hops = 0
    successful_runs = 0
    total_correct_choices = 0
    total_steps = 0
    all_actuals = []  
    all_predictions = []  

    nodes_with_features = [node for node in G.nodes if "features" in G.nodes[node]]

    for run in range(num_runs):
        if len(nodes_with_features) < 2:
            print("Not enough nodes with features for evaluation.")
            break

        source_node, target_node = random.sample(nodes_with_features, 2)
        print(f"\nRun {run + 1}/{num_runs}: Source {source_node} -> Target {target_node}")

        path, correct_choices, steps, step_actuals, step_predictions = find_path(model, G, source_node, target_node, max_hops)

        # Print path
        print(f"  Path: {path}")

        if path and path[-1] == target_node:
            print(f"  Path found in {steps} steps. Correct choices: {correct_choices}/{steps}")
            successful_runs += 1
            total_hops += steps
        else:
            print(f"  Path failed after {steps} steps.")

        all_actuals.extend(step_actuals)
        all_predictions.extend(step_predictions)

        total_correct_choices += correct_choices
        total_steps += steps

    success_rate = (successful_runs / num_runs) * 100 if num_runs > 0 else 0
    average_hops = total_hops / successful_runs if successful_runs > 0 else float('inf')
    accuracy = total_correct_choices / total_steps if total_steps > 0 else 0.0
    precision = precision_score(all_actuals, all_predictions, zero_division=0)
    recall = recall_score(all_actuals, all_predictions, zero_division=0)
    f1 = f1_score(all_actuals, all_predictions, zero_division=0)

    # Print summary
    print(f"\n--- Summary ---")
    print(f"Success rate: {success_rate:.2f}% ({successful_runs}/{num_runs})")
    print(f"Average hops: {average_hops:.2f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-score: {f1:.2f}")

    return success_rate, average_hops, accuracy, precision, recall, f1

success_rate, average_hops, accuracy, precision, recall, f1 = evaluate_pathfinding(model, G, max_hops=20, num_runs=100)

Feature vectors and splits saved successfully.
Feature vectors and splits loaded successfully.
Training set size: 565446, Test set size: 141362
Training the SVM model with a linear kernel for interpretability...
Model training complete.
SVM model saved to 'trained_model_svm.pkl'.
Evaluating the model on the test set...
Test Accuracy: 0.9910
Test Precision: 0.0000
Test Recall: 0.0000
Test F1: 0.0000
SVM model loaded successfully from 'trained_model_svm.pkl'.

Run 1/100: Source 30031 -> Target 12276
  Path: ['30031', '10475', '24641', '3566', '8122', '58328', '5421', '2393', '21243', '2393', '28750', '27027', '26066', '28272', '27526', '541', '9689', '10436', '178', '28002', '9849']
  Path failed after 20 steps.

Run 2/100: Source 16844 -> Target 27357
  Path: ['16844', '11039', '13845', '24509', '10634', '13110', '130743', '17287', '16198', '11339', '8410', '15754', '15165', '1855921', '13218', '14942', '9807', '17152', '19879', '11985', '11889']
  Path failed after 20 steps.

Run 3/100

  Path: ['23290', '1178202', '383796', '1178202', '279212', '27757', '1769473', '395836', '24721', '25801', '22858', '17892', '21048', '146', '27165', '1657', '10757', '12937', '26082', '75', '5816']
  Path failed after 20 steps.

Run 33/100: Source 22126 -> Target 9301
  Path: ['22126', '14263', '82823', '29521', '17999', '17152', '9807', '14258', '11813', '14150', '12344', '15866', '10625', '12796', '11087', '16399', '17287', '16198', '11339', '8410', '15754']
  Path failed after 20 steps.

Run 34/100: Source 25090 -> Target 3859
  Path: ['25090', '109008', '4695', '2998', '23185', '20839', '6256', '3339', '26473', '3339', '649', '10387', '12786', '4076', '20683', '16462', '10780', '29073', '4907', '18563', '3346']
  Path failed after 20 steps.

Run 35/100: Source 9996 -> Target 22126
  Path: ['9996', '15474', '26622', '21914', '884', '24116', '19587', '29754', '6629', '6536', '25271', '17083', '7067', '18818', '12878', '5666', '14109', '5666', '1235018', '5666', '12878']
  Path fail

  Path: ['23595', '27384', '6871', '25954', '6871', '19560', '19722', '19601', '29840', '40710', '26569', '15638', '11893', '24140', '29864', '29056', '373935', '24802', '19560', '2183', '19560']
  Path failed after 20 steps.

Run 65/100: Source 12046 -> Target 28570
  Path: ['12046', '6256', '20839', '23185', '2998', '4695', '16811', '28788', '540', '9689', '10436', '178', '28002', '9849', '23317', '8109', '2079', '541', '866', '872', '21665']
  Path failed after 20 steps.

Run 66/100: Source 28640 -> Target 22586
  Path: ['28640', '782922', '901453', '1104702', '916459', '1158563', '951482', '663683', '897246', '745333', '1128033', '1708672', '1112924', '1828622', '748640', '340651', '739987', '636174', '1321537', '636174', '699704']
  Path failed after 20 steps.

Run 67/100: Source 5572 -> Target 16332
  Path: ['5572', '524006', '25679', '16638', '21925', '28198', '119820', '222344', '26004', '222344', '172408', '922414', '7199', '18311', '27074', '3649', '581', '10868', '168419', '

  Path: ['23940', '9072', '3339', '26473', '3339', '6256', '20839', '23185', '2998', '4695', '16811', '28788', '540', '9689', '10436', '178', '28002', '9849', '23317', '8109', '2079']
  Path failed after 20 steps.

Run 98/100: Source 17157 -> Target 20505
  Path: ['17157', '5975', '866', '872', '21665', '11802', '854', '30030', '11247', '11156', '1057', '13666', '8901', '7543', '5421', '2393', '21243', '53845', '7614', '541', '9689']
  Path failed after 20 steps.

Run 99/100: Source 13647 -> Target 3421
  Path: ['13647', '9845', '25275', '5712', '3171', '1183', '2783', '5851', '13600', '9301', '5629', '9750', '7779', '14882', '6431', '28634', '4735', '2006', '540', '9689', '10436']
  Path failed after 20 steps.

Run 100/100: Source 28730 -> Target 3144
  Path: ['28730', '184223', '168005', '178', '28002', '9849', '23317', '8109', '2079', '541', '3144']
  Path found in 10 steps. Correct choices: 5/10

--- Summary ---
Success rate: 5.00% (5/100)
Average hops: 5.80
Accuracy: 0.38
Precisio

In [3]:
print(success_rate, average_hops, accuracy, precision, recall, f1)

5.0 5.8 0.383618455158113 1.0 0.383618455158113 0.5545147995503934


In [4]:
success_rate1, average_hops1, accuracy1, precision1, recall1, f11 = evaluate_pathfinding(model, G, max_hops=20, num_runs=1000)


Run 1/1000: Source 13504 -> Target 2291
  Path: ['13504', '16545', '14258', '11813', '14150', '12344', '15866', '10625', '12796', '11087', '16399', '17287', '16198', '11339', '8410', '15754', '15165', '15754', '11522', '9815', '11734']
  Path failed after 20 steps.

Run 2/1000: Source 11487 -> Target 29813
  Path: ['11487', '283', '6943', '3405', '6943', '1213', '18608', '21382', '27354', '27898', '541', '9689', '10436', '178', '28002', '9849', '23317', '8109', '2079', '10277', '25773']
  Path failed after 20 steps.

Run 3/1000: Source 26792 -> Target 18331
  Path: ['26792', '13477', '29176', '1426089', '1894912', '761188', '337919', '1393268', '1018401', '1393268', '337919', '761188', '337919', '761188', '337919', '1393268', '337919', '1018401', '1393268', '337919', '1018401']
  Path failed after 20 steps.

Run 4/1000: Source 21481 -> Target 25695
  Path: ['21481', '14750', '19535', '20186', '3649', '27074', '18311', '6106', '10000', '10571', '28401', '4547', '13362', '5471', '11298'

  Path: ['4561', '27083', '3859', '8752', '9589', '758', '8683', '22869', '22087', '3339', '26473', '1277630', '1234377', '1684738', '20839', '23185', '2998', '4695', '16811', '28788', '540']
  Path failed after 20 steps.

Run 33/1000: Source 10429 -> Target 10838
  Path: ['10429', '66852', '27951', '541', '9689', '10436', '178', '28002', '9849', '23317', '8109', '2079', '10277', '25773', '4445', '4480', '10881', '3339', '26473', '95293', '14571']
  Path failed after 20 steps.

Run 34/1000: Source 14517 -> Target 24955
  Path: ['14517', '17047', '14517', '8410', '11339', '16198', '17287', '16399', '10625', '12796', '11087', '12260', '16108', '19109', '12539', '27595', '16544', '11236', '16681', '10593', '616']
  Path failed after 20 steps.

Run 35/1000: Source 24222 -> Target 7645
  Path: ['24222', '27083', '3859', '8752', '9589', '758', '8683', '22869', '22087', '3339', '26473', '2084757', '381862', '4907', '18563', '3346', '4076', '20683', '16462', '10780', '29073']
  Path failed aft

  Path: ['27193', '28605', '11893', '26569', '15638', '31159', '15638', '26569', '6941', '19722', '19560', '6871', '25954', '6871', '27384', '24932', '7725', '15033', '29902', '25833', '9805']
  Path failed after 20 steps.

Run 65/1000: Source 21979 -> Target 27988
  Path: ['21979', '1521724', '2846', '15537', '18528', '5471', '13362', '11290', '4547', '28401', '6106', '18311', '27074', '3649', '581', '10868', '581', '16128', '17609', '26588', '2001']
  Path failed after 20 steps.

Run 66/1000: Source 3104 -> Target 698
  Path: ['3104', '12968', '160104', '8479', '1877', '7759', '15072', '5499', '3195', '4957', '3857', '17802', '2087', '15816', '130236', '11298', '5471', '13362', '11290', '4547', '28401']
  Path failed after 20 steps.

Run 67/1000: Source 21474 -> Target 5816
  Path: ['21474', '19760', '9605', '10464', '13268', '3859', '27083', '4561', '8357', '19624', '4894', '1115438', '14935', '540', '9689', '10436', '178', '28002', '9849', '23317', '8109']
  Path failed after 20 st

  Path: ['26662', '8557', '7078', '8300', '3476', '7196', '19401', '6891', '4076', '20683', '16462', '10780', '29073', '4907', '18563', '3346', '9923', '26213', '3365', '3924', '8853']
  Path failed after 20 steps.

Run 97/1000: Source 8391 -> Target 10572
  Path: ['8391', '4076', '20683', '16462', '10780', '29073', '4907', '18563', '3346', '9923', '26213', '3365', '3924', '8853', '10067', '3233', '25272', '1444233', '801425', '3339', '26473']
  Path failed after 20 steps.

Run 98/1000: Source 16193 -> Target 12620
  Path: ['16193', '5230', '1895', '28580', '26596', '15171', '110356', '15171', '141417', '15171', '676609', '454205', '99700', '3622', '7484', '27354', '21382', '4074', '540', '9689', '10436']
  Path failed after 20 steps.

Run 99/1000: Source 22811 -> Target 17152
  Path: ['22811', '231103', '13477', '29176', '315034', '1172360', '1249292', '1835310', '1210689', '411235', '1311497', '568977', '123709', '27103', '249814', '18216', '19561', '13666', '1057', '11156', '11247']

  Path: ['25978', '23747', '7257', '1268', '23321', '1527', '10034', '23001', '13585', '801', '806', '1532', '12549', '24133', '20109', '28969', '3339', '26473', '95293', '14571', '21382']
  Path failed after 20 steps.

Run 130/1000: Source 10910 -> Target 18535
  Path: ['10910', '12398', '11033', '16108', '12260', '8410', '11339', '16198', '17287', '16399', '10625', '12796', '11087', '11607', '11971', '15866', '11813', '14150', '12344', '16990', '14258']
  Path failed after 20 steps.

Run 131/1000: Source 14818 -> Target 29966
  Path: ['14818', '12505', '10003', '16198', '17287', '16399', '8410', '11339', '16429', '14873', '10590', '27390', '11590', '12230', '10778', '26127', '25620', '12144', '16545', '14258', '11813']
  Path failed after 20 steps.

Run 132/1000: Source 7071 -> Target 7763
  Path: ['7071', '15044', '8431', '5471', '13362', '11290', '4547', '28401', '6106', '18311', '27074', '3649', '581', '10868', '168419', '6806', '4872', '1258', '4731', '1323', '7585']
  Path faile

...

  Path: ['22388', '17802', '3857', '8240', '13268', '3859', '27083', '4561', '8357', '19624', '4894', '217779', '1115438', '14935', '540', '9689', '10436', '178', '28002', '9849', '23317']
  Path failed after 20 steps.

Run 861/1000: Source 19024 -> Target 27638
  Path: ['19024', '77092', '297916', '14707', '10778', '12230', '5363', '15914', '10590', '27390', '11590', '11889', '10057', '10377', '10625', '12796', '11087', '16399', '17287', '16198', '11339']
  Path failed after 20 steps.

Run 862/1000: Source 12094 -> Target 17292
  Path: ['12094', '12739', '12614', '11691', '11817', '22232', '10625', '12796', '11087', '16399', '17287', '16198', '11339', '8410', '15754', '15165', '334508', '14446', '20800', '13469', '14140']
  Path failed after 20 steps.

Run 863/1000: Source 12446 -> Target 22590
  Path: ['12446', '67602', '3649', '27074', '18311', '6106', '10000', '10571', '28401', '4547', '13362', '5471', '11298', '178', '28002', '9849', '23317', '8109', '2079', '541', '9689']
  Path 

  Path: ['4939', '8420', '13393', '5810', '1985723', '27728', '21895', '30030', '11247', '11156', '1057', '13666', '8901', '7543', '5421', '2393', '21243', '784161', '178', '28002', '9849']
  Path failed after 20 steps.

Run 893/1000: Source 17665 -> Target 11691
  Path: ['17665', '20093', '10764', '20093', '612', '13235', '26613', '14942', '9807', '17152', '19879', '11985', '11889', '10057', '10377', '10625', '12796', '11087', '16399', '17287', '16198']
  Path failed after 20 steps.

Run 894/1000: Source 12585 -> Target 12828
  Path: ['12585', '13504', '16545', '14258', '11813', '14150', '12344', '15866', '10625', '12796', '11087', '16399', '17287', '16198', '11339', '8410', '15754', '15165', '138894', '1969473', '138894']
  Path failed after 20 steps.

Run 895/1000: Source 12874 -> Target 21589
  Path: ['12874', '2079', '8109', '23317', '9849', '28002', '178', '11298', '5471', '13362', '11290', '4547', '28401', '6106', '18311', '27074', '3649', '581', '10868', '581', '16128']
  Path 

  Path: ['28036', '10590', '27390', '11590', '12230', '10778', '8410', '11339', '16198', '17287', '16399', '10625', '12796', '11087', '12260', '16108', '19109', '12539', '27595', '16544', '11236']
  Path failed after 20 steps.

Run 926/1000: Source 12555 -> Target 28537
  Path: ['12555', '9271', '28465', '7543', '5421', '2393', '21243', '784161', '178', '28002', '9849', '23317', '8109', '2079', '541', '9689', '10436', '24222', '27083', '3859', '8752']
  Path failed after 20 steps.

Run 927/1000: Source 23245 -> Target 7335
  Path: ['23245', '188210', '465789', '4181', '202845', '15683', '28579', '965931', '14571', '21382', '27354', '27898', '541', '9689', '10436', '178', '28002', '9849', '23317', '8109', '2079']
  Path failed after 20 steps.

Run 928/1000: Source 7088 -> Target 28580
  Path: ['7088', '10040', '4368', '9805', '24395', '19722', '19560', '6871', '25954', '6871', '27384', '24932', '7725', '15033', '29902', '25833', '28438', '25582', '14920', '24179', '25644']
  Path failed

  Path: ['21382', '27354', '27898', '541', '9689', '10436', '178', '28002', '9849', '23317', '8109', '2079', '10277', '25773', '4445', '4480', '10881', '3339', '26473', '118819', '2291']
  Path failed after 20 steps.

Run 957/1000: Source 12796 -> Target 4480
  Path: ['12796', '10625', '10397', '15598', '17287', '16198', '11339', '8410', '15754', '15165', '15754', '11522', '9815', '11734', '12630', '12340', '10057', '11889', '17152', '9807', '14258']
  Path failed after 20 steps.

Run 958/1000: Source 3083 -> Target 22341
  Path: ['3083', '625914', '61291', '9228', '6190', '3649', '27074', '18311', '6106', '10000', '10571', '28401', '4547', '13362', '5471', '11298', '178', '28002', '9849', '23317', '8109']
  Path failed after 20 steps.

Run 959/1000: Source 10057 -> Target 20791
  Path: ['10057', '11889', '10625', '12796', '11087', '16399', '17287', '16198', '11339', '8410', '15754', '15165', '1855921', '13218', '14942', '9807', '17152', '19879', '11985', '10590', '27390']
  Path faile

  Path: ['8095', '541', '9689', '10436', '178', '28002', '9849', '23317', '8109', '2079', '10277', '25773', '4445', '4480', '10881', '3339', '26473', '2084757', '118819', '2291', '9923']
  Path failed after 20 steps.

Run 989/1000: Source 23324 -> Target 18196
  Path: ['23324', '4695', '2998', '23185', '20839', '6256', '3339', '26473', '3339', '649', '10387', '12786', '4076', '20683', '16462', '10780', '29073', '4907', '18563', '3346', '9923']
  Path failed after 20 steps.

Run 990/1000: Source 3849 -> Target 1455
  Path: ['3849', '16462', '4076', '20683', '9923', '26213', '3365', '3924', '8853', '10067', '3233', '25272', '164904', '554063', '1527', '23321', '23001', '10034', '806', '801', '25606']
  Path failed after 20 steps.

Run 991/1000: Source 12007 -> Target 26140
  Path: ['12007', '13130', '13774', '15637', '15697', '1195901', '15637', '10175', '16198', '17287', '16399', '8410', '11339', '16429', '14873', '10590', '27390', '11590', '12230', '10778', '26127']
  Path failed after

In [6]:
print(success_rate1, average_hops1, accuracy1, precision1, recall1, f11)

4.6 9.826086956521738 0.40420847839442964 1.0 0.40420847839442964 0.5757100667225726
